In [1]:
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
"""
INCOMPLETE, NOT READY TO BE RUN
"""
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten
from modeling_utils import NumpyDataGenerator


# Parameters

def train_model(metadata_location,
                model_architecture = None,
                data_dim=(128,128),
                batch_size=64,
                n_classes=10,
                training_folds = [10,2,3,4,5,6,7,8,9],
                validation_folds = [1],
                shuffle=True):
    """
    """
    params = {'dim': data_dim,
              'batch_size': batch_size,
              'n_classes': n_classes,
              'shuffle': shuffle}

    # Datasets
    metadata = pd.read_csv(metadata_location)
    if shuffle:
        metadata = metadata.sample(frac=1)
    id_to_file_mapping = dict(zip(metadata['fsID'],metadata['location']))
    train_data = metadata[metadata['fold'].isin(training_folds)]
    test_data = metadata[metadata['fold'].isin(validation_folds)]
    # Generators
    training_generator = NumpyDataGenerator(list(train_data['fsID']), list(train_data['classID']),id_to_file_mapping, **params)
    validation_generator = NumpyDataGenerator(list(test_data['fsID']), list(test_data['classID']), id_to_file_mapping, **params)

    # Design model
    model = Sequential()
    model.add(Dense(12, input_shape=(128,128), activation='relu',name='main_input'))
    model.add(Dense(8, activation='relu',name='middle_layer'))
    model.add(Flatten())
    model.add(Dense(n_classes, activation='sigmoid',name='classifier'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # Train model on dataset
    steps_per_epoch = np.ceil(len(metadata) / batch_size)
    validation_steps = np.ceil(len(list(test_data['classID']))/batch_size)
    model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,
                        workers=6,
                        verbose=1,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps,
                        epochs=1,
                        shuffle=True
                       )
    #Validate
    validation_generator_2 = NumpyDataGenerator(list(test_data['fsID']), list(test_data['classID']), id_to_file_mapping, **params)
    
    predictions = model.predict_generator(validation_generator_2,
                                      steps = validation_steps,
                                      verbose=True)
    y_pred = np.argmax(predictions, axis=1)
    print('Confusion Matrix')
    cm = confusion_matrix(list(test_data['classID']), y_pred[:len(list(test_data['classID']))])
    print(cm)
    print(np.unique(np.array(y_pred), return_counts=True))
    print(np.unique(np.array(test_data['classID']), return_counts=True))
    return y_pred, test_data['classID']

Using TensorFlow backend.


In [3]:
metadata_file = "/media/romulo/6237-3231/urban_sound_challenge/spectrograms_128_array/location_mapping.csv"

In [4]:
train_model(metadata_file)

W0714 20:27:02.826447 139800350508864 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 20:27:02.849943 139800350508864 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 20:27:02.857152 139800350508864 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0714 20:27:02.962707 139800350508864 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer i

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (Dense)           (None, 128, 12)           1548      
_________________________________________________________________
middle_layer (Dense)         (None, 128, 8)            104       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
classifier (Dense)           (None, 10)                10250     
Total params: 11,902
Trainable params: 11,902
Non-trainable params: 0
_________________________________________________________________
None
986.0
Epoch 1/1
101/101 [==============================] - 3s 31ms/step
Confusion Matrix
[[ 31   0   5   8 155 134   1 462   4   0]
 [  7   0   1   0  21  31   0  73   0   0]
 [ 34   1   3   7 127 157   0 469   2   0]
 [ 37   1   6   6 127 116   0 341   1   0]
 [ 2